1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

        *Наименование вакансии
        *Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
        *Ссылку на саму вакансию        
        *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

2)Задание похожее, но подход к сбору данных здесь другой.
Необходимо собрать информацию по продуктам питания с сайтов: https://rskrf.ru/ratings/produkty-pitaniya/ и https://roscontrol.com/category/produkti/ Получившийся список должен содержать:

        *Наименование продукта
        *Категорию продукта        (например Бакалея)
        *Подкатегорию продукта    (апример "Рис круглозерный")
        *Параметр "безопасность"
        *Параметр "качество"
        *Общий балл
        *Сайт, откуда получена информация
Данная структура должна быть одинаковая для продуктов с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

Можно выполнить по своему желанию одно любое задание или оба при желании и возможности.

## Задание 1:

### hh.ru

#### Импорт необходимых библиотек:

In [5]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import time

#### Ввод названия вакансии:

In [91]:
vacancy = input("Введите название вакансии:")
vacancy = re.sub('\+',"%2B",vacancy);
vacancy = re.sub('#',"%23",vacancy);
vacancy = re.sub(' +',"+",vacancy);

Введите название вакансии:программист с++


#### Шаблоны минимальной и максимальной зарплаты:

In [92]:
sal_min_1 = re.compile('от [0-9]+')
sal_min_2 = re.compile('[0-9]+\-')
sal_max_1 = re.compile('до [0-9]+')
sal_max_2 = re.compile('\-[0-9]+')

#### Создание запроса и получение ответа:

In [93]:
main_link = 'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

In [94]:
vacansy_list = []
salary_list = []
min_salary = []
max_salary = []
url_list = []
source_name_list = []
employer_list = []
pages = 10

In [95]:
for pg_num in range(0,pages):
    # создаем запрос и получаем ответ
    page = str(pg_num);
    html = requests.get(main_link + vacancy + '&page=' + page,headers=headers)
    # проверяем доступность страницы
    if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
        break    
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    # ищем блок с вакансиями
    vacansy_block = bsObj.findAll(class_="bloko-link HH-LinkModifier")
    # создаем список вакансий
    for i,vacansy in enumerate(vacansy_block):
        vacansy_list.append(vacansy_block[i].getText())
    # создаем список информационных элементов по вакансиям
    item_info_block = bsObj.findAll(class_="vacancy-serp-item__row vacancy-serp-item__row_header")
    # создаем список элементов с информациям по зарплатам
    for i,salary in enumerate(item_info_block):
        salary_list.append(item_info_block[i].findChildren()[-1].getText())
        salary_list[i] = re.sub('\xa0',"",salary_list[i])
    # создаем список нижней границы зарплат
    for salary in salary_list:
        if re.findall(sal_min_1,salary):
            min_salary.append(re.findall(sal_min_1,salary))        
        elif re.findall(sal_min_2,salary):
            min_salary.append(re.findall(sal_min_2,salary))
        else:
            min_salary.append('')    
    # создаем список верхней границы зарплат   
    for salary in salary_list:
        if re.findall(sal_max_1,salary):
            max_salary.append(re.findall(sal_max_1,salary))        
        elif re.findall(sal_min_2,salary):
            max_salary.append(re.findall(sal_max_2,salary))
        else:
            max_salary.append('')    
    # создаем список ссылок на вакансию 
    for i,url in enumerate(item_info_block):
        if item_info_block[i].findChildren()[-3].get('href') is not None:
            url_list.append(item_info_block[i].findChildren()[-3].get('href'))    
        else:
            url_list.append(item_info_block[i].findChildren()[-3].findChildren()[0].get('href'))
    # создаем список ссылок на источник данных
    for i,info in enumerate(item_info_block):
        source_name_list.append('hh.ru')
    # ищем блок с работодателями
    employer_block = bsObj.findAll(class_="bloko-link bloko-link_secondary")
    # создаем список работодателей
    for i,employer in enumerate(employer_block):
        employer_list.append(employer_block[i].getText())
    time.sleep(1);

In [96]:
# чистим список нижней границы от ненужных символов    
for i,salary in enumerate(min_salary):
    if salary is not None:
        min_salary[i] = re.sub('от ',"",str(min_salary[i]))
        min_salary[i] = re.sub('-',"",str(min_salary[i]))
        min_salary[i] = min_salary[i][2:-2]
# чистим список верхней границы от ненужных символов           
for i,salary in enumerate(max_salary):
    if salary is not None:
        max_salary[i] = re.sub('до ',"",str(max_salary[i]))
        max_salary[i] = re.sub('-',"",str(max_salary[i]))  
        max_salary[i] = max_salary[i][2:-2]

In [97]:
s_vacansy_list = pd.Series(vacansy_list)
s_min_salary = pd.Series(min_salary)
s_max_salary = pd.Series(max_salary)
s_url_list = pd.Series(url_list)
s_source_name_list = pd.Series(source_name_list)
s_employer_list = pd.Series(employer_list)

In [98]:
df = pd.DataFrame()
df['vacansy'] = s_vacansy_list
df['min_salary'] = s_min_salary
df['max_salary'] = s_max_salary
df['url'] = s_url_list
df['source'] = s_source_name_list
df['employer'] = s_employer_list
vacancy = re.sub('\+'," ",vacancy);
vacancy = re.sub('%2B',"+",vacancy);
vacancy = re.sub('%23',"#",vacancy);
df.to_csv('hh ' + vacancy + '.csv',index=False,header=True, encoding='utf-8')

In [99]:
vacancy

'программист с++'

In [100]:
df.head(10)

,vacansy,min_salary,max_salary,url,source,employer
0,Программист С++/Qt,70000,,https://hh.ru/vacancy/31273030?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
1,Программист С++ (графика),80000,,https://hh.ru/vacancy/32543357?query=%D0%BF%D1...,hh.ru,Cappasity Inc.
2,Программист С++,,100000,https://hh.ru/vacancy/32417827?query=%D0%BF%D1...,hh.ru,Группа Компаний «АМЕ»
3,Ведущий программист С++,200000,,https://hh.ru/vacancy/32773076?query=%D0%BF%D1...,hh.ru,"Эверис, Представительство"
4,Программист С++,,,https://hh.ru/vacancy/30288124?query=%D0%BF%D1...,hh.ru,ЛАНТЕР (ГК ЛАНИТ)
5,Программист Junior С++,60000,,https://hh.ru/vacancy/33254957?query=%D0%BF%D1...,hh.ru,"Прикладная Логистика, АНО НИЦ CALS-Технологий"
6,Программист С++,,150000,https://hh.ru/vacancy/33514106?query=%D0%BF%D1...,hh.ru,МСК БЛ ГРУПП
7,Программист С++,,,https://hh.ru/vacancy/32759566?query=%D0%BF%D1...,hh.ru,Paradigm Geophysical
8,Программист С++,,,https://hh.ru/vacancy/32591713?query=%D0%BF%D1...,hh.ru,ООО НТ Прогресс
9,Программист С++ blockchain,,,https://hh.ru/vacancy/32152698?query=%D0%BF%D1...,hh.ru,Руформ


### Superjob

#### Ввод названия вакансии:

In [101]:
vacancy = input("Введите название вакансии:")
vacancy = re.sub('\+',"%2B",vacancy);
vacancy = re.sub('#',"%23",vacancy);
vacancy = re.sub(' +',"%20",vacancy);

Введите название вакансии:программист c#


#### Главный url и user agent:

In [102]:
main_link = 'https://www.superjob.ru/vacancy/search/?keywords=' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

#### Создаем необходимые структуры и шаблоны:

In [103]:
vacansy_list=[]
salary_list=[]
min_salary=[]
max_salary=[]
max_salary=[]
url_list=[]
source_name_list=[]
employer_list=[]
pages = 10

In [104]:
sal_min_1 = re.compile('от[0-9]+₽')
sal_min_2 = re.compile('[0-9]+—')
sal_min_3 = re.compile('[0-9]+₽')
sal_max_1 = re.compile('^[0-9]+₽')
sal_max_2 = re.compile('—[0-9]+₽')
url_re = re.compile('^icMQ_ _1QIBo f-test-link')
employer_re = re.compile('^icMQ_ _205Zx f-test-link')

In [105]:
for pg_num in range(1,pages + 1):
     # создаем запрос и получаем ответ
    page = str(pg_num);
    html = requests.get(main_link + vacancy + '&page=' + page,headers=headers)
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    # создаем блок с вакансиями
    vacansy_block = bsObj.findAll(class_="_3mfro CuJz5 PlM3e _2JVkc _3LJqf")
    if vacansy_block == []:        
        print('На странице '+ main_link + vacancy + '&page=' + page + ' нет вакансий')
        break;
    # создаем список вакансий
    for vacansy in vacansy_block:
        vacansy_list.append(vacansy.getText())
    # создаем блок зарплат
    salary_block = bsObj.findAll(class_="_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz")
    # создаем список зарплат
    for i,salary in enumerate(salary_block):
        salary_list.append(salary.getText()) 
        salary_list[i] = re.sub('\xa0',"",salary_list[i])
    # создаем список нижней границы зарплаты
    for salary in salary_list:
        if re.findall(sal_min_1,salary):
            min_salary.append(re.findall(sal_min_1,salary))        
        elif re.findall(sal_min_2,salary):
            min_salary.append(re.findall(sal_min_2,salary))
        elif re.findall(sal_min_3,salary):
            min_salary.append(re.findall(sal_min_3,salary))
        else:
            min_salary.append('')   
    # создаем список верхней границы зарплаты
    for salary in salary_list:
        if re.findall(sal_max_1,salary):
            max_salary.append(re.findall(sal_max_1,salary))        
        elif re.findall(sal_max_2,salary):
            max_salary.append(re.findall(sal_max_2,salary))
        else:
            max_salary.append('')   
    # создаем блок url
    url_block = bsObj.findAll(class_=url_re)
    # создаем список url
    for i,url in enumerate(url_block):
        url_list.append('https://www.superjob.ru' + url_block[i].get('href'))
    # создаем список ссылок на источник данных
    for i,info in enumerate(vacansy_block):
        source_name_list.append('superjob.ru')
    # создаем блок работодателей
    employer_block = bsObj.findAll(class_=employer_re)
    # создаем список работодателей
    for i,employer in enumerate(employer_block):
        employer_list.append(employer_block[i].getText())
    time.sleep(1);

In [106]:
# чистим список нижней границы от ненужных символов    
for i,salary in enumerate(min_salary):
    if salary is not None:
        min_salary[i] = re.sub('от',"",str(min_salary[i]))
        min_salary[i] = re.sub('—',"",str(min_salary[i]))
        min_salary[i] = re.sub('₽',"",str(min_salary[i]))
        min_salary[i] = min_salary[i][2:-2]
# чистим список верхней границы от ненужных символов    
for i,salary in enumerate(max_salary):
    if salary is not None:
        max_salary[i] = re.sub('—',"",str(max_salary[i]))
        max_salary[i] = re.sub('₽',"",str(max_salary[i]))
        max_salary[i] = max_salary[i][2:-2]

In [107]:
s_vacansy_list2 = pd.Series(vacansy_list)
s_min_salary2 = pd.Series(min_salary)
s_max_salary2 = pd.Series(max_salary)
s_url_list2 = pd.Series(url_list)
s_source_name_list2 = pd.Series(source_name_list)
s_employer_list2 = pd.Series(employer_list)

In [112]:
df2 = pd.DataFrame()
df2['vacansy'] = s_vacansy_list2
df2['min_salary'] = s_min_salary2
df2['max_salary'] = s_max_salary2
df2['url'] = s_url_list2
df2['source'] = s_source_name_list2
df2['employer'] = s_employer_list2
vacancy = re.sub('%20'," ",vacancy);
vacancy = re.sub('%2B',"+",vacancy);
vacancy = re.sub('%23',"#",vacancy);
df2.to_csv('superjob ' + vacancy + '.csv',index=False,header=True, encoding='utf-8')

In [111]:
df2.head(10)

,vacansy,min_salary,max_salary,url,source,employer
0,Инженер-программист / Разработчик C#,300000,,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,ФГУ НПП Гамма
1,Инженер-программист C#,,,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,Центральная ППК
2,.NET Developer / Разработчик C#,,,https://www.superjob.ru/vakansii/net-developer...,superjob.ru,Информационные технологии
3,Программист-разработчик C#,,,https://www.superjob.ru/vakansii/programmist-r...,superjob.ru,АйТи Элемент
4,Unity/C# Game Developer,,,https://www.superjob.ru/vakansii/unity-3240922...,superjob.ru,Tinybuild
5,Unity/C# Porting Developer (Netherlands),,,https://www.superjob.ru/vakansii/unity-3240882...,superjob.ru,Tinybuild
6,Senior Web.NET C#/F# разработчик,180000,,https://www.superjob.ru/vakansii/senior-web-32...,superjob.ru,Открытая Клиника
7,Web-программист / Web-разработчик,150000,200000,https://www.superjob.ru/vakansii/web-programmi...,superjob.ru,Минерал-Инфо
8,Инженер-программист,98310,98310,https://www.superjob.ru/vakansii/inzhener-prog...,superjob.ru,"""Корпорация ""ВНИИЭМ"""
9,Программист C#,160000,,https://www.superjob.ru/vakansii/programmist-c...,superjob.ru,Сеть медицинских клиник «ИММА» («IMMA»)


## Задание 2:

In [6]:
main_link = 'https://rskrf.ru/ratings/produkty-pitaniya/' 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

In [7]:
products_url_list=[]
subproducts_url_list=[]
product_name_list=[]
category_name_list=[]
subcategory_name_list=[]
name_url_list=[]

In [8]:
html = requests.get(main_link,headers=headers)
if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
else:
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    # ищем блок с ссылками на категории продуктов
    products_url_block = bsObj.findAll(class_="category-item")

#### Получаем список ссылок на все категории продуктов:

In [9]:
for i, item in enumerate(products_url_block):
    products_url_list.append('https://rskrf.ru' +  products_url_block[i].find('a').get('href'))

#### Получаем список ссылок на все подкатегории продуктов:

In [10]:
for url in products_url_list:
    link = url;
    html = requests.get(link)
    if html.status_code is not requests.codes.ok:
        print('Страница ' + html.url + ' не доступна')
        break
    # делаем объект bs
    bsObj = bs(html.text,'lxml')
    subproducts_url_block = bsObj.findAll(class_="category-item")
    for i, item in enumerate(subproducts_url_block):
        subproducts_url_list.append('https://rskrf.ru' +  subproducts_url_block[i].find('a').get('href'))

#### Импортируем selenium:

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By

#### Список имен продуктов:

In [12]:
product_name_list=[]

In [13]:
driver = webdriver.PhantomJS('C:\\Users\\krilatiy\\phantomjs-2.1.1-windows\\bin\\phantomjs')
for url in subproducts_url_list:
    driver.get(url)
    time.sleep(3)
    content = driver.find_elements_by_xpath('//*[@class="col-xl-4 col-lg-4 col-md-4 product"]')
    for elem in content:
        product_name_list.append(elem.text)
driver.close()

D:\Anaconda3\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [14]:
product_name_list[:10]

['Globus\nСухие продукты экструзионной технологии (СПЭТ) - шарики кукурузные, шоколадные\n4.513\n24.99 ₽ /100 г',
 'Schär\nЗавтраки сухие: шарики шоколадные Milly Magic\n4.330\n186.8 ₽ /100 г',
 'Кузя Лакомкин\nСухие завтраки. Шарики мультизерновые, глазированные шоколадные, «Кузя Лакомкин»\n4.244\n41.39 ₽ /100 г',
 'Krosby choco balls\nЗавтраки сухие, экструзионные, из зернового сырья, обогащенные витаминами и минеральными веществами, глазированные шарики шоколадные\n4.230\n41 ₽ /100 г',
 'Коровка\nЗавтраки сухие, фигурные, глазированные. Шарики, вкус шоколада\n4.086\n35.79 ₽ /100 г',
 'Любятово\nЗавтраки готовые: шарики шоколадные\n4.040\n17.98 ₽ /100 г',
 'Metro cheff\nШарики со вкусом шоколада\n3.405\n21.78 ₽ /100 г',
 'Mamori\nЗавтраки сухие Crunchy тм Mamori: шарики шоколадные, хрустящие\n3.004\n38 ₽ /100 г',
 'Ого!\nСухие завтраки «Десять витаминов» Choca (шарики со вкусом молочного шоколада)\n2.916\n29.99 ₽ /100 г',
 'Каждый день\nСухие завтраки глазированные, без добавления ви

#### Ссылки на все продукты:

In [ ]:
driver = webdriver.PhantomJS('C:\\Users\\krilatiy\\phantomjs-2.1.1-windows\\bin\\phantomjs')
for url in subproducts_url_list:
    driver.get(url)
    time.sleep(3)
    content = driver.find_elements_by_xpath('//*[@class="col-xl-4 col-lg-4 col-md-4 product"]/div[2]/a')
    for elem in content:
        name_url_list.append(elem.get_attribute("href"))
driver.close()

In [1011]:
name_url_list[:10]

['https://rskrf.ru/goods/sukhie-produkty-ekstruzionnoy-tekhnologii-spet-shariki-kukuruznye-shokoladnye/',
 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/#',
 'https://rskrf.ru/goods/zavtraki-sukhie-shariki-shokoladnye-milly-magic/',
 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/#',
 'https://rskrf.ru/goods/sukhie-zavtraki-shariki-multizernovye-glazirovannye-shokoladnye-kuzya-lakomkin/',
 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/#',
 'https://rskrf.ru/goods/zavtraki-sukhie-ekstruzionnye-iz-zernovogo-syrya-obogashchennye-vitaminami-i-mineralnymi-veshchestva/',
 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/#',
 'https://rskrf.ru/goods/zavtraki-sukhie-figurnye-glazirovannye-shariki-vkus-shokolada/',
 'https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/#']

#### Удаляем якоря:

In [18]:
nul = [v for i, v in enumerate(name_url_list) if i % 2 == 0]

In [19]:
len(product_name_list)==len(nul)

True

#### Попытка получить названия категорий:

In [22]:
link='https://rskrf.ru/goods/sukhie-produkty-ekstruzionnoy-tekhnologii-spet-shariki-kukuruznye-shokoladnye/'
html2 = requests.get(link,headers=headers)
html2.encoding = 'utf-8'
if html2.status_code is not requests.codes.ok:
        print('Страница ' + html2.url + ' не доступна')
else:
    # делаем объект bs
    bsObj2 = bs(html2.text,'lxml')
    # ищем блок с ссылками на категории продуктов
    category_name_block = bsObj.findAll(class_="breadcrumb-item")

In [23]:
category_name_block

[<li class="breadcrumb-item"><a href="/">ÐÐ»Ð°Ð²Ð½Ð°Ñ</a></li>,
 <li class="breadcrumb-item"><a href="/ratings/">Ð ÐµÐ¹ÑÐ¸Ð½Ð³Ð¸</a></li>,
 <li class="breadcrumb-item"><a href="/ratings/produkty-pitaniya/">ÐÑÐ¾Ð´ÑÐºÑÑ Ð¿Ð¸ÑÐ°Ð½Ð¸Ñ</a></li>,
 <li aria-current="page" class="breadcrumb-item active">Ð¥Ð»ÐµÐ±Ð¾Ð±ÑÐ»Ð¾ÑÐ½ÑÐµ Ð¸Ð·Ð´ÐµÐ»Ð¸Ñ</li>]

#### Почистим имена продуктов:

In [1012]:
for i,name in enumerate(product_name_list):
    product_name_list[i] = re.sub('\n'," ",product_name_list[i])
    product_name_list[i] = re.sub('\d{1}\.\d{3}',"",product_name_list[i])
    product_name_list[i] = re.sub('\d+\.\d+ ₽',"",product_name_list[i])
    product_name_list[i] = re.sub('\d+ ₽',"",product_name_list[i])
    product_name_list[i] = re.sub('/\d+ г',"",product_name_list[i])
    product_name_list[i] = re.sub('/кг',"",product_name_list[i])
    product_name_list[i] = re.sub(' кг$',"",product_name_list[i])
    product_name_list[i] = re.sub('/\d+г',"",product_name_list[i])
    product_name_list[i] = re.sub('/шт',"",product_name_list[i])
    product_name_list[i] = re.sub('/л',"",product_name_list[i])
    product_name_list[i] = re.sub(' \d+г$',"",product_name_list[i])
    product_name_list[i] = re.sub(' +$',"",product_name_list[i])

#### Функция для поиска тега содержащего текст tagText по url в классе rating-item:

In [1039]:
def findTagNum(tagText,url):
    tags=[]
    driver = webdriver.PhantomJS('C:\\Users\\krilatiy\\phantomjs-2.1.1-windows\\bin\\phantomjs')
    driver.get(url)
    content = driver.find_elements_by_xpath('//*[@class="rating-item"]')
    for elem in content:
        tags.append(elem.text)
    driver.close()
    for i,tag in enumerate(tags):
        tags[i] = re.sub('\n',"",tags[i])
        tags[i] = re.sub('\d{1}\.\d{3}',"",tags[i])
        if tags[i] == tagText:
            return i+1
    return None

In [1020]:
nul

['https://rskrf.ru/goods/sukhie-produkty-ekstruzionnoy-tekhnologii-spet-shariki-kukuruznye-shokoladnye/',
 'https://rskrf.ru/goods/zavtraki-sukhie-shariki-shokoladnye-milly-magic/',
 'https://rskrf.ru/goods/sukhie-zavtraki-shariki-multizernovye-glazirovannye-shokoladnye-kuzya-lakomkin/',
 'https://rskrf.ru/goods/zavtraki-sukhie-ekstruzionnye-iz-zernovogo-syrya-obogashchennye-vitaminami-i-mineralnymi-veshchestva/',
 'https://rskrf.ru/goods/zavtraki-sukhie-figurnye-glazirovannye-shariki-vkus-shokolada/',
 'https://rskrf.ru/goods/zavtraki-gotovye-shariki-shokoladnye/',
 'https://rskrf.ru/goods/shariki-so-vkusom-shokolada/',
 'https://rskrf.ru/goods/zavtraki-sukhie-crunchy-tm-mamori-shariki-shokoladnye-khrustyashchie/',
 'https://rskrf.ru/goods/sukhie-zavtraki-desyat-vitaminov-choca-shariki-so-vkusom-molochnogo-shokolada/',
 'https://rskrf.ru/goods/sukhie-zavtraki-glazirovannye-bez-dobavleniya-vitaminov-sladkie-shokoladnye-shariki/',
 'https://rskrf.ru/goods/zavtraki-sukhie-khlopya-kukuruz

#### Заходим на страницу каждого товара и заполняем поля: качество, безопасность, общий рейтинг:

In [1050]:
quality_list=[]
rate_list=[]
safety_list=[]
for url in nul:
    html = requests.get(url,headers=headers)
    bsObj = bs(html.text,'lxml')
    index = findTagNum('Качество',url)
    if index is not None:
        quality = bsObj.findAll(class_="rating-item")[index].findChildren()[2].getText()
    else:
        quality = 'None'    
    index = findTagNum('Безопасность',url)
    if index is not None:
        safety = bsObj.findAll(class_="rating-item")[index].findChildren()[2].getText()  
    else:
        safety = 'None'
    rate = bsObj.find(class_="rating-item big").findChildren()[2].getText()
    quality_list.append(quality)    
    rate_list.append(rate)
    safety_list.append(safety)
    time.sleep(1)

In [1]:
s_pnl = pd.Series(product_name_list)
s_rate = pd.Series(rate_list)
s_nul = pd.Series(nul)
s_quality = pd.Series(quality_list)
s_safety = pd.Series(safety_list)

NameError: name 'pd' is not defined

In [145]:
df = pd.DataFrame()
df['product_name'] = s_pnl
df['quality'] = s_quality
df['safety'] = s_safety
df['rate'] = s_rate
df['product_url'] = s_nul
df.to_csv('products.csv',index=False,header=True, encoding='utf-8')